In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import warnings

import tensorflow as tf
import keras
from keras import utils
from keras.layers import Layer
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, CSVLogger
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import InceptionResNetV2, ResNet50
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from keras import backend
from keras import layers
from keras import models
from keras import utils as keras_utils

import pandas as pd
from sklearn.metrics import roc_curve, auc
from scipy import interp
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from itertools import cycle
import pickle
from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter
from sklearn import metrics
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from PIL import Image
import glob
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


--------------------------

In [7]:
keras.backend.clear_session()

model = ResNet50(include_top=False,
                 weights='imagenet',
                 input_tensor=None,
                 input_shape=(512,512,3),
                 pooling=None,
                 classes=30)

# model = InceptionResNetV2(include_top=True, 
#                           weights=None, 
#                           input_tensor=None, 
#                           input_shape=(512,512,6), 
#                           pooling=None, 
#                           classes=30)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy', ROC_AUC, PR_AUC])

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 518, 518, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 256, 256, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 256, 256, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [13]:
#load and shuffle x/y ids
x_id, y_id = np.load("../../data/x_id_pos_control.npy"), np.load("../../data/y_id_pos_control.npy")

np.random.shuffle(x_id)
training_id, val_id = x_id[:int(0.85*len(x_id))], x_id[int(0.85*len(x_id)):]
num_training_samples = len(training_id)
num_val_samples = len(val_id)

print("Number of training samples: ", num_training_samples)
print("Number of validation samples: ", num_val_samples)

Number of training samples:  6704
Number of validation samples:  1184


In [14]:
# Parameters
params = {'dim': (512,512),
          'batch_size': 4,
          'n_classes': 30,
          'n_channels': 6,
          'shuffle': True}

id_dict = {'train': training_id,
           'val': val_id}

# Generators
training_generator = DataGenerator(id_dict['train'], **params)
validation_generator = DataGenerator(id_dict['val'], **params)

In [15]:
checkpointer = ModelCheckpoint('../saved_models/model_ckpnt_inception_resnet_072019.weights', 
                               monitor='val_categorical_accuracy', 
                               verbose=0, 
                               save_best_only=True, 
                               save_weights_only=True, 
                               mode='max', 
                               period=1)

logger = CSVLogger('../logs/training_inception_resnet_072019.log', 
                   separator=',', 
                   append=False)

In [19]:
# Train model on dataset
model.fit_generator(generator=training_generator,
                    steps_per_epoch=num_training_samples // params['batch_size'],
                    epochs=25,
                    verbose=1,
                    callbacks=[checkpointer, logger],
                    validation_data=validation_generator,
                    validation_steps=num_val_samples // params['batch_size'],
                    class_weight=None,
                    max_queue_size=20,
                    workers=6,
                    use_multiprocessing=True,
                    shuffle=True,
                    initial_epoch=0)

Epoch 1/25
  82/1676 [>.............................] - ETA: 9:12 - loss: 4.8403 - categorical_accuracy: 0.0274 - ROC_AUC: 0.0000e+00 - PR_AUC: 0.0000e+00

Process ForkPoolWorker-32:
Process ForkPoolWorker-27:
Process ForkPoolWorker-36:
Process ForkPoolWorker-31:
Process ForkPoolWorker-25:
Process ForkPoolWorker-33:
Process ForkPoolWorker-29:
Process ForkPoolWorker-28:
Process ForkPoolWorker-30:
Process ForkPoolWorker-35:
Process ForkPoolWorker-26:
Process ForkPoolWorker-34:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/media/ryan/SATA1/anaconda3/envs/keras-gpu/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/media/ryan/SATA1/anaconda3/envs/keras-gpu/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/media/ryan/SATA1/anaconda3/envs/keras-gpu/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/media/ryan/SATA1/anaconda3/envs/ker

KeyboardInterrupt: 

  File "/media/ryan/SATA1/anaconda3/envs/keras-gpu/lib/python3.6/site-packages/numpy/lib/npyio.py", line 422, in load
    fid = open(os_fspath(file), "rb")
  File "/media/ryan/SATA1/anaconda3/envs/keras-gpu/lib/python3.6/site-packages/numpy/lib/npyio.py", line 447, in load
    pickle_kwargs=pickle_kwargs)
KeyboardInterrupt
  File "/media/ryan/SATA1/anaconda3/envs/keras-gpu/lib/python3.6/site-packages/numpy/lib/format.py", line 712, in read_array
    array = numpy.fromfile(fp, dtype=dtype, count=count)
KeyboardInterrupt


In [ ]:
df = pd.read_csv('../logs/training_072019.log')
df

## Evaluation

In [ ]:
model.load_weights('../saved_models/model_ckpnt_072019_weights.h5')

In [ ]:
y_pred = model.predict_generator(generator=validation_generator,
                                 steps=num_val_samples // params['batch_size'],
                                 max_queue_size=20,
                                 workers=6,
                                 use_multiprocessing=True,
                                 verbose=1)

In [ ]:
y_test = np.empty(shape=(val_id.shape[0], params['n_classes']))

for i in range(y_test.shape[0]):
    y_test[i,:] = np.load('../../data/train_parsed_pos_control/y' + y_id[i][1:] + '.npy')

In [ ]:
print("y_test shape: ", y_test.shape)
print("y_pred shape: ", y_pred.shape)

In [ ]:
plot_precision_recall(y_test, y_pred)

In [ ]:
plot_roc(y_test, y_pred)

### Custom metrics for classification measurement

In [5]:
def ROC_AUC(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred,
                          num_thresholds=100,
                          curve='ROC',
                          summation_method='trapezoidal')[0]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [6]:
def PR_AUC(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred,
                          num_thresholds=100,
                          curve='PR',
                          summation_method='careful_interpolation')[0]
    K.get_session().run(tf.local_variables_initializer())
    return auc

### Model/function definitions

In [5]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, self.n_classes), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load('../../data/train_parsed_pos_control/' + ID + '.npy')

            # Store class
            y[i,] = np.load('../../data/train_parsed_pos_control/y' + ID[1:] + '.npy')

        return X, y

In [6]:
def plot_precision_recall(y_true, y_pred):
    precision = dict()
    recall = dict()
    average_precision = dict()
    for i in range(params['n_classes']):
        precision[i], recall[i], _ = precision_recall_curve(y_test[:, i],
                                                            y_pred[:, i])
        average_precision[i] = average_precision_score(y_test[:, i], y_pred[:, i])

    # A "micro-average": quantifying score on all classes jointly
    precision["micro"], recall["micro"], _ = precision_recall_curve(y_test.ravel(),
        y_pred.ravel())
    average_precision["micro"] = average_precision_score(y_test, y_pred,
                                                         average="micro")

    colors = cycle(['navy', 'turquoise', 'darkorange', 'cornflowerblue', 'teal'])

    plt.figure(figsize=(15, 15))
    f_scores = np.linspace(0.2, 0.8, num=4)
    lines = []
    labels = []
    for f_score in f_scores:
        x = np.linspace(0.01, 1)
        y = f_score * x / (2 * x - f_score)
        l, = plt.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2)
        plt.annotate('f1={0:0.1f}'.format(f_score), xy=(0.9, y[45] + 0.02))

    lines.append(l)
    labels.append('iso-f1 curves')
    l, = plt.plot(recall["micro"], precision["micro"], color='gold', lw=2)
    lines.append(l)
    labels.append('micro-average Precision-recall (area = {0:0.2f})'
                  ''.format(average_precision["micro"]))

    for i, color in zip(range(params['n_classes']), colors):
        l, = plt.plot(recall[i], precision[i], color=color, lw=2)
        lines.append(l)
        labels.append('Precision-recall for class {0} (area = {1:0.2f})'
                      ''.format(i, average_precision[i]))

    fig = plt.gcf()
    fig.subplots_adjust(bottom=0.25)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Multi-class Precision-Recall curve')
    plt.legend(lines, labels, loc=(1.1, 0), prop=dict(size=14))

    plt.show()

In [7]:
def plot_roc(y_true, y_pred):
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(params['n_classes']):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_pred.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(params['n_classes'])]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(params['n_classes']):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= params['n_classes']

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.figure(figsize=(15,15))
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(params['n_classes']), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=1)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Multi-class Receiver operating characteristic')
    plt.legend(loc=(1.1, 0), prop=dict(size=14))
    plt.show()